In [9]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [10]:
import pandas as pd
import sqlalchemy as sa
from lib import fetch_SPARQL as fs

In [5]:
# Read a sample dataset of 44107 cities from wikidata with their population
wikidata_query = """
SELECT ?city ?cityLabel ?population
WHERE {
  ?city wdt:P31/wdt:P279* wd:Q515.
  OPTIONAL { ?city wdt:P1082 ?population. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

df_wiki_path = fs.fetch_and_store(wikidata_query, 'query_cities_population_entityid.csv')
# read predownloaded dataset. This is a sample of 44107 cities from wikidata with their population
df_wiki = pd.read_csv("query_cities_population_entityid.csv")

# Sort dataframe in ascending order by population
df_wiki.sort_values(by=['population'], ascending=True, inplace=True)
df_wiki.head()

,city,population,cityLabel
71713,http://www.wikidata.org/entity/Q12385031,0,Campairos
23513,http://www.wikidata.org/entity/Q721338,0,Greenhorn
55549,http://www.wikidata.org/entity/Q1991598,0,South Park View
55560,http://www.wikidata.org/entity/Q1992792,0,North Mankato
73477,http://www.wikidata.org/entity/Q12393950,0,Q12393950


In [14]:
# MusicBrainz database credentials
HOST = "localhost"
DATABASE = "musicbrainz_db"
PASSWORD = "musicbrainz"
USER = "musicbrainz"
PORT = 5432

sa_conn_str = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = sa.create_engine(sa_conn_str)

In [15]:
with engine.connect() as conn:
    query = '''
    SELECT a.name as name, at.name as type
    FROM area AS a 
    LEFT JOIN area_type as at 
    ON a.type = at.id;
    '''
    
    df_musicbrainz = pd.read_sql(sa.text(query), conn)

# Fetches all ~120k areas from the MusicBrainz database with their area type
df_musicbrainz

,name,type
0,Russia,Country
1,United States,Country
2,Sankt-Peterburg,City
3,Atlanta,City
4,Cleveland,City
...,...,...
16977,Ampfing,Municipality
16978,Oberschützen,City
16979,Eberbach,City
16980,Cockermouth,City


In [16]:
# Convert all names to lowercase for comparision
df_musicbrainz['name'] = df_musicbrainz['name'].str.lower()
df_wiki['cityLabel'] = df_wiki['cityLabel'].str.lower()

In [17]:
not_in_MeB = df_wiki[~df_wiki.cityLabel.isin(df_musicbrainz.name)]
is_in_MeB = df_wiki[df_wiki.cityLabel.isin(df_musicbrainz.name)]

In [18]:
# Previewing cities that are NOT in MusicBrainz database
not_in_MeB.sort_values(by = 'population', ascending=False)

,city,population,cityLabel
915,http://www.wikidata.org/entity/Q215646,http://www.wikidata.org/.well-known/genid/87cd...,edo
31520,http://www.wikidata.org/entity/Q215646,http://www.wikidata.org/.well-known/genid/87cd...,edo
104617,http://www.wikidata.org/entity/Q26368,9999,schotten
77696,http://www.wikidata.org/entity/Q1614791,9999,kolno
84180,http://www.wikidata.org/entity/Q855011,9999,heves
...,...,...,...
105874,http://www.wikidata.org/entity/Q16508929,NaN,mandji
105876,http://www.wikidata.org/entity/Q21294265,NaN,akanda
105877,http://www.wikidata.org/entity/Q23730961,NaN,puerto carreño
105878,http://www.wikidata.org/entity/Q32143142,NaN,san andrés
